# k-Nearest-Neighbour classifiers
Here we use the k-nearest-neighbor method on the converted data from the descriptive_analysis.ipynb.
As reference we use dataMining_R_8_ml.Rmd (./uebung/knn_uebung/) from the lecture.

In [50]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [ ]:
data = pd.read_csv('./data/converted_test_2020_06_24_14-47-10.csv')
train = pd.read_csv('./data/converted_train_2020_06_24_14-47-10.csv')

In [44]:
# Encode route abb column
data['route_abb']=encoder.fit_transform(data['route_abb'])
train['route_abb']=encoder.fit_transform(train['route_abb'])

# Encode Request_AM_PM column
data['Request_AM_PM']=encoder.fit_transform(data['Request_AM_PM'])
train['Request_AM_PM']=encoder.fit_transform(train['Request_AM_PM'])

# Encode min_future_price
train['min_future_price_in_eur']=encoder.fit_transform(train['min_future_price_in_eur'])

In [45]:
# Drop columns we do not need from data
data_dropped = data.drop(columns=['Request_Date', 'Flight_Date', 'flight_unique_id', 'flight_number', 'Currency', 'Price', 'Request_Weekday', 'Departure_hour'])
data_dropped = data_dropped.loc[:, ~data_dropped.columns.str.contains('^Unnamed')]

# Drop columns we do not need from train
train_dropped = train.drop(columns=['Request_Date', 'Flight_Date', 'flight_unique_id', 'flight_number', 'Currency', 'Price', 'min_future_price', 'Request_Weekday', 'Departure_hour'])
train_dropped = train_dropped.loc[:, ~train_dropped.columns.str.contains('^Unnamed')]

In [46]:
print(data_dropped.head())
print(train_dropped.head())

Duration_Time  route_abb  price_in_eur  Request_AM_PM
0          692.0          2     23.811657              0
1          680.0          2     34.968663              1
2          668.0          2     35.016015              0
3          656.0          2     35.016015              1
4          644.0          2     34.973793              0
   Duration_Time  route_abb  price_in_eur  min_future_price_in_eur  \
0           56.0          3    208.070000                     4021   
1           44.0          3    259.070000                     4021   
2           32.0          3    259.070000                     4021   
3           20.0          3    259.070000                     4021   
4           58.0          2    112.979088                     3926   

   Request_AM_PM  
0              0  
1              1  
2              0  
3              1  
4              0  


In [47]:
# Generate test and target / Figure out which features to drop. 'min_future_price_in_eur' needs to be dropped!
X = train_dropped.drop(columns=['min_future_price_in_eur'])
y = train_dropped['min_future_price_in_eur'].values
X

,Duration_Time,route_abb,price_in_eur,Request_AM_PM
0,56.0,3,208.070000,0
1,44.0,3,259.070000,1
2,32.0,3,259.070000,0
3,20.0,3,259.070000,1
4,58.0,2,112.979088,0
...,...,...,...,...
83619,959.0,3,35.690000,0
83620,947.0,3,46.830000,1
83621,935.0,3,46.830000,0
83622,923.0,3,39.690000,1


In [48]:
# Split dataset into train and test data (might want to try random_state=0 to randomize each calculation)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)
print(X_train.shape)
print(y_train.shape)

(50174, 4)
(50174,)


In [49]:
# We try to find the best value for k.
k_list = np.arange(1,20)
train_accuracy = np.empty(len(k_list))
test_accuracy = np.empty(len(k_list))

for i,k in enumerate(k_list):
    knn_test = KNeighborsClassifier(n_neighbors=k)
    knn_test.fit(X_train, y_train)
    train_accuracy[i] = knn_test.score(X_train, y_train)
    test_accuracy[i] = knn_test.score(X_test, y_test) 

In [51]:
#Generate plot
plt.plot(k_list, test_accuracy, label='Testing Accuracy')
plt.plot(k_list, train_accuracy, label='Training accuracy')
plt.legend()
plt.xlabel('Number of neighbors')
plt.ylabel('Accuracy (Cross−Validation)')
plt.show()

KeyboardInterrupt: 

In [34]:
knn = KNeighborsClassifier(n_neighbors=15)
#Fit the model
knn.fit(X_train,y_train)
#Get accuracy
knn.score(X_test,y_test)

0.07294469357249626